<a href="https://colab.research.google.com/github/dmlcks123/minkim1423.github.io/blob/main/labs_0406.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# EDA(탐색적 자료분석)와 LInear regression
# 와인분류예측 이후에 레드와인 화이트와인 분류도?

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
#1. 데이터 전처리, EDA

# 데이터 구조: 레드와인 + 화이트와인 append

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
url2 = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"

In [ ]:
redwine = pd.read_csv(url, sep = ";", header= 0)

#식별변수 추가
redwine["type"] = "red"
redwine.head()

In [ ]:
whitewine = pd.read_csv(url2, sep = ";", header=0)

whitewine["type"] = "white"
whitewine.head()

In [ ]:
redwine.count()

In [ ]:
whitewine.count()

In [ ]:
# append

wine = redwine.append(whitewine)
wine.shape

In [ ]:
# Vinho Verde사의 Minho지방 와인데이터
# petillance(스파클?)

# fixed acidity	고정산, 와인의 산도를 제어											
# volatile acidity	휘발산, 와인의 향과 연관											
# citric acid	구연산, 와인의 신선함을 올림											
# residual sugar	잔여 당분, 와인의 단 맛 올림											
# chlorides	염화물, 와인의 짠 맛의 원인											
# free sulfur dioxide,	S02(이산화황), 미생물 증식 및 와인 산화 방지 											
# total sulfur dioxide,	SO2(이산화황), 총 이산화황 --> free S02 + bound SO2 조합											
# density	밀도, 와인의 무게감을 나타냄											
# pH	산성도, 와인의 신 맛의 정도											
# sulphates	황 산화염, 와인 산화 방지 											
# alcohol	알코올, 와인의 단 맛과 무게감에 영향											
# quality	와인의 질	3,4,5,6,7,8

In [ ]:
# summary

wine.describe() 

In [ ]:
# rename

wine.columns = wine.columns.str.replace(" ", "_")
wine.head()

In [ ]:
# histogram
sns.distplot(wine.quality, hist = True, kde = False, color = "green", label = "wine quality")

In [ ]:
# 종류별 histogram

# subsample
red_q = wine.loc[wine["type"] == "red", "quality"]
white_q = wine.loc[wine["type"] == "white", "quality"]

# distplot
sns.set_style("darkgrid")
sns.distplot(red_q, norm_hist = True, kde = False, color = "Red", label = "red wine")
sns.distplot(white_q, norm_hist = True, kde = False, color = "Blue", label = "white wine")
plt.title("Distibution of Quality Wine Type")
plt.xlabel("Quality Score")
plt.ylabel("Density")
plt.legend();

# 와인품질점수가 중간에 몰려 있음

In [ ]:
# wine 종류별 summary
wine.groupby("type")["quality"].describe() 

In [ ]:
#2 와인품질예측: 선형회귀모형
#예측능력 평가지표: RMSE(root mean square error), R_square

In [ ]:
# model class
from sklearn.linear_model import LinearRegression
lr = LinearRegression(fit_intercept= True)

In [ ]:
# 특징행렬과 대상벡터
X = wine.drop(["type", "quality"], axis = 1)
y = wine.quality

In [ ]:
# train, test set 분류
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
# random_state는 시드값, test size default는 0.25

In [ ]:
# fitting
lr.fit(X_train, y_train)

In [ ]:
# prediction
y_pred = lr.predict(X_test)
y_train_pred = lr.predict(X_train)

In [ ]:
# regression, p-value

X_train = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train).fit()
model.summary()

In [ ]:
# test set kdensity

sns.distplot(y_pred, hist = False, kde = True, color = "red", label = "y_pred")
sns.distplot(y_test, hist = False, kde = True, color = "blue", label = "y")
plt.xlabel("Quality Score")
plt.ylabel("Density")
plt.legend();


In [ ]:
# R^2
print("train data\'s rsquare : ", format(lr.score(X_train, y_train)))
print("test data\'s rsquare ; ", format(lr.score(X_test, y_test)))

In [ ]:
# RMSE
print("train RMSE is ", np.round(np.sqrt(mean_squared_error(y_train, y_train_pred)),2))
print("test RMSE is ", np.round(np.sqrt(mean_squared_error(y_test, y_pred)),2))

In [ ]:
## 전진 선택법: p-value 기준이하이면 모델에 포함

variables = X_train.columns.tolist() ## 설명 변수 리스트
 
y = y_train ## 반응 변수
selected_variables = [] ## 선택된 변수들
sl_enter = 0.05 # p-value, 유의수준 0.05 이하면 선택안함#####
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        X = X_train[selected_variables+[col]]
        X = sm.add_constant(X)
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [ ]:
selected_variables

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 15
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
plt.plot(steps,adjusted_r_squared, marker='o')
    
plt.ylabel('Adjusted R Squared',fontsize=font_size)
plt.grid(True)
plt.show()

In [ ]:
## 후진 소거법
variables = X_train.columns.tolist() ## 설명 변수 리스트
 
y = y_train ## 반응 변수
selected_variables = variables ## 초기에는 모든 변수가 선택된 상태
sl_remove = 0.05 # p-value 값이 유의수준 5% 보다 크다면 변수제거
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(selected_variables) > 0:
    X = sm.add_constant(X_train[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
    max_pval = p_vals.max() ## 최대 p-value
    if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
 
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [ ]:
selected_variables

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 15
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
plt.plot(steps,adjusted_r_squared, marker='o')
    
plt.ylabel('Adjusted R Squared',fontsize=font_size)
plt.grid(True)
plt.show()

In [ ]:
## 전진 단계별 선택법: 유의수준보다 작으면 선택 이후 모델적합
## 모델 적합후 변수 중 가장큰 p-value 유의수준보다 크면 삭제

variables = X_train.columns.tolist() ## 설명 변수 리스트
 
y = y_train ## 반응 변수
selected_variables = [] ## 선택된 변수들
sl_enter = 0.05 # 선택기준 유의수준
sl_remove = 0.05 # 제거기준 유의수준
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        X = X_train[selected_variables+[col]]
        X = sm.add_constant(X)
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        ## 선택된 변수들에대해서
        ## 어떤 변수를 제거할지 고른다.
        while len(selected_variables) > 0:
            selected_X = X_train[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
            max_pval = selected_pval.max()
            if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                remove_variable = selected_pval.idxmax()
                selected_variables.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [ ]:
selected_variables

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 15
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
plt.plot(steps,adjusted_r_squared, marker='o')
    
plt.ylabel('Adjusted R Squared',fontsize=font_size)
plt.grid(True)
plt.show()

In [ ]:
model.summary()
# 전진선택, 후진선택 출처:https://zephyrus1111.tistory.com/65